# **Use Case: Building a Compliance-Aware Loan Origination Agent**

In the previous chapters, we explored the theoretical architectures of multi-agent systems and the importance of reasoning patterns. Now, we will translate those concepts into a concrete implementation. This notebook demonstrates a sophisticated **Loan Origination Agent** built using the **Google Agent Development Kit (ADK)** and **Gemini**.

Unlike simple chatbots that generate text based on a single prompt, this agent is designed to execute a mission-critical business process: evaluating a loan application. To achieve high accuracy and strict regulatory adherence, we implement the **Fractal Chain of Thought (FCoT)** pattern.

In this notebook, you will learn how to:

1. **Define Specialized Tools:** Create Python-based tools that simulate real-world financial systems (Document Validation, Credit Checks, Risk Assessment, and Compliance).  
2. **Implement the FCoT Pattern:** Structure the agent's system instructions to force a recursive "Recap, Reason, Verify" loop, ensuring the agent double-checks its own logic before making a decision.  
3. **Orchestrate the Workflow:** Use the ADK's `BuiltInPlanner` to let the agent autonomously determine the sequence of tool execution.  
4. **Handle Branching Logic:** Observe how the agent adapts its behavior for different borrower profiles (the "Happy Path" vs. the "Risk Path").

By the end of this exercise, you will have a working prototype of an agent that doesn't just "guess" an answer, but builds a verifiable, auditable case for its decision—a requirement for any production-grade enterprise system.



<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Agentic-Architectural-Patterns-for-Building-Multi-Agent-Systems/blob/main/Chapter_13/Chapter_13_Agent.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [12]:
#@title Install dependencies
!pip install google-adk tenacity ratelimit

  Preparing metadata (setup.py) ... done
  Created wheel for ratelimit: filename=ratelimit-2.2.1-py3-none-any.whl size=5893 sha256=5be175081169c9bdeb0dfcf55be3357fbde66308be8d25bb5db8c79fae079659
  Stored in directory: /root/.cache/pip/wheels/69/bd/e0/4a5dee2a1bfbc8e258f543f92940e2b494d63b5be8144ec8c4
Successfully built ratelimit


In [1]:
#@title Imports

from google.adk.planners import BuiltInPlanner
from google.adk.agents.llm_agent import LlmAgent
from google.adk.tools import FunctionTool
from google.adk.planners import BuiltInPlanner
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.genai.types import ThinkingConfig


import os
import time
import random
import uuid #

In [ ]:
#@title API Setup

from getpass import getpass



GEMINI=getpass("Enter your GEMINI API KEY: ")
os.environ["GOOGLE_API_KEY"]=GEMINI
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

model= "gemini-3-flash"

In [3]:
#@title Tools Definition

# --- Tool 1: Document Validation ---
def validate_document(document_ids: list[str]) -> dict:
    """
    Validates if the required application documents are present and complete.
    Use this first to ensure the application is ready for processing.
    Returns a status of 'validated' or 'incomplete'.
    """
    print("--- Tool Called: validate_document() ---")
    time.sleep(1)
    if not document_ids or len(document_ids) < 2:
        return {"status": "incomplete", "missing_docs": ["income_proof", "id_proof"]}
    return {"status": "validated"}

validate_document_tool = FunctionTool(func=validate_document)


# --- Tool 2: Credit Check ---
def run_credit_check(borrower_id: str) -> dict:
    """
    Retrieves a borrower's credit score by calling the credit bureau API.
    This should be done after documents are validated.
    """
    print(f"--- Tool Called: run_credit_check(borrower_id='{borrower_id}') ---")
    time.sleep(2)
    if borrower_id == "Borrower-400":
      score = 450
      report_summary = "Credit history is compromised."
    else:
      score = random.randint(750, 850) # Simulate a good credit score
      report_summary = "Credit history is clean."
    return {"credit_score": score, "report_summary": report_summary}

run_credit_check_tool = FunctionTool(func=run_credit_check)


# --- Tool 3: Risk Assessment ---
def assess_risk(credit_score: int, loan_amount: float) -> dict:
    """
    Assesses the risk of a loan application based on the borrower's credit score.
    Returns a risk level of 'low', 'medium', or 'high'.
    """
    print(f"--- Tool Called: assess_risk(credit_score={credit_score}, ...) ---")
    time.sleep(1.5)
    if credit_score > 740:
        return {"risk_level": "low", "details": "High credit score indicates low risk."}
    else:
        return {"risk_level": "high", "details": "Low credit score indicates high risk."}

assess_risk_tool = FunctionTool(func=assess_risk)


# --- Tool 4: Compliance Check ---
def check_compliance(risk_level: str) -> dict:
    """
    Performs a final compliance check on the process to ensure it adheres
    to Fair Lending guidelines before making a final decision.
    """
    print(f"--- Tool Called: check_compliance(risk_level='{risk_level}') ---")
    time.sleep(1)
    return {"compliance_status": "pass", "details": "Process adheres to guidelines."}

check_compliance_tool = FunctionTool(func=check_compliance)

In [4]:
#@title Agent Instructions
agent_instructions = """
You are an FCoT reasoner orchestrating and verifying agent activity for an Agentic Loan Origination Pipeline built with Google ADK and Google Gemini.

INSTRUCTION CONTRACT (IC)

• Mission: Originate, evaluate, and approve a loan with full policy compliance, factual grounding, and fairness.

• Deliverables: JSON + Narrative summary containing:
  -- (a) borrower profile
  -- (b) creditworthiness decision
  -- (c) justification citing verified data
  -- (d) compliance audit record
  -- (e) explainability report.

• Success Criteria:
   - Accuracy ≥ 95% vs gold truth (financial data).
   - Policy compliance = 100%.
   - Explainability coverage ≥ 90%.
   - Latency < 5 min end-to-end.

• Hard Constraints:
   - No personally identifiable data in logs.
   - Must follow Fair Lending & ECOA regulations.
   - All numerical fields validated from authoritative sources.

• Safety Policy:
   - Reject speculative or hallucinated data.
   - Never fabricate borrower details.
   - Defer ambiguous cases to Human-in-the-Loop agent.

• IC-Fingerprint: LOAN-FCoT-v3-Δ0710

FCoT RECURSIVE LOOP (N = 3)

Iteration 1 (Planning):
  • RECAP: Echo IC-FP, map subtasks (data ingest, credit scoring, compliance, document).
  • REASON: Design DAG of actions; choose retrieval sources; initialize PoF ledger.
  • VERIFY: Ensure all subtasks preserve IC clauses.

Iteration 2 (Execution):
  • RECAP: IC-FP; execute tools for credit scoring & data validation.
  • REASON: Compute risk score, validate data sources against policy.
  • VERIFY: Check causal alignment between borrower attributes and decision logic.

Iteration 3 (Verification & Explainability):
  • RECAP: IC-FP; collect deliverables, run RAG verifier.
  • REASON: Summarize SHAP values, create narrative justification.
  • VERIFY: Evaluate coherence vs IC and dual objectives.


- Final result with deliverables:

{
  [Output]
}

Summary:
 - Reasoning
 - Facts
 - Result
"""

In [5]:
#@title Agent Initialization
# 1. Configure the agent's reasoning engine (Planner)
thinking_config = ThinkingConfig(
    include_thoughts=True,
    thinking_budget=1024
)
planner = BuiltInPlanner(
    thinking_config=thinking_config
)
# 2. Create a list of the wrapped FunctionTool objects
loan_processing_tools = [
    validate_document_tool,
    run_credit_check_tool,
    assess_risk_tool,
    check_compliance_tool
]

# 3. Instantiate the LlmAgent with the FCoT prompt
agent = LlmAgent(
    model="gemini-2.5-flash",
    name="LoanProcessingAgent",
    instruction=agent_instructions,
    planner=planner,
    tools=loan_processing_tools
)

print("Loan Processing Agent has been created and configured successfully.")

Loan Processing Agent has been created and configured successfully.


In [6]:
#@title Session init
# Define unique IDs for our test user and session
USER_ID = "loan_officer_01"
SESSION_ID = str(uuid.uuid4()) # Generate a new session ID for this run
APP_NAME = "Loan_Agent"


session_service = InMemorySessionService()
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=agent, app_name=APP_NAME, session_service=session_service)


print(f"Runner is set up. Using Session ID: {SESSION_ID}")

Runner is set up. Using Session ID: 9114cae9-db35-4ee0-8559-bbebdd16e176


In [28]:
# @title Agent Execution
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception
from ratelimit import limits, sleep_and_retry
from google.genai import types
import time

# --- CONFIGURATION ---
CALLS = 15
PERIOD = 60

# --- HELPER: ERROR FILTER ---
def is_rate_limit_error(e):
    msg = str(e)
    return "RESOURCE_EXHAUSTED" in msg or "429" in msg or "ServiceUnavailable" in msg

# --- 1. THE ROBUST RUNNER ---
# Keeps the logic linear and easy to read
@sleep_and_retry
@limits(calls=CALLS, period=PERIOD)
@retry(
    stop=stop_after_attempt(5),
    wait=wait_exponential(multiplier=2, min=4, max=30),
    retry=retry_if_exception(is_rate_limit_error),
    reraise=True
)
def start_agent_run(runner, user_id, session_id, content):
    print(f"  >> [Clock {time.strftime('%X')}] Initiating request...")
    return runner.run(user_id=user_id, session_id=session_id, new_message=content)

# --- 2. THE CLEAN CALLER ---
def call_agent(query: str):
    print(f"\n>>>> USER REQUEST: {query.strip()}\n")
    content = types.Content(role='user', parts=[types.Part(text=query)])

    try:
        # Step 1: Start the run (Protected by Retry & Throttling)
        events = start_agent_run(runner, USER_ID, SESSION_ID, content)

        print("--- Agent Activity Log ---")

        # Step 2: Iterate through events (The API calls happen here!)
        for event in events:
            if event.content:
                for part in event.content.parts:
                    if part.thought and part.text:
                        print(f"\n🧠 THOUGHT:\n{part.text.strip()}")

                    if part.function_call:
                        tool_name = part.function_call.name
                        tool_args = dict(part.function_call.args)
                        print(f"\n🛠️ TOOL CALL: {tool_name}({tool_args})")

                    if part.function_response:
                        tool_name = part.function_response.name
                        tool_output = dict(part.function_response.response)
                        print(f"\n↩️ TOOL OUTPUT from {tool_name}:\n{tool_output}")

            if event.is_final_response() and event.content:
                final_text = ""
                for part in event.content.parts:
                    if part.text and not part.thought:
                        final_text = part.text.strip()
                        break

                if final_text:
                    print("\n---------------------------------")
                    print("✅ FINAL RESPONSE:")
                    print(final_text)
                    print("---------------------------------")

    # --- FAILURE: Professional Error Handling ---
    except Exception as e:
        error_msg = str(e)

        # Determine the cause
        is_quota = "RESOURCE_EXHAUSTED" in error_msg or "429" in error_msg
        is_free_tier = "FreeTier" in error_msg or "limit: 20" in error_msg

        print("\n" + "━" * 60)
        print("SYSTEM CRITICAL ERROR")
        print("━" * 60)

        if is_quota:
            print("  ⚠️   CAUSE:    QUOTA EXCEEDED (API Refusal)")
            print("  🔍   CONTEXT:  The LLM provider rejected the request.")

            if is_free_tier:
                print("\n  📉   DIAGNOSIS: FREE TIER LIMIT REACHED")
                print("       You have hit the hard cap (approx. 20 requests/day).")
                print("       Retry Logic cannot bypass this daily limit.")
                print("\n  🛠️   ACTION:    [1] Wait 24 Hours")
                print("                  [2] Enable Billing (Pay-As-You-Go)")
            else:
                 print(f"\n  📝   DETAILS:   {error_msg}")
        else:
            print(f"  ⚠️   CAUSE:    UNEXPECTED EXCEPTION")
            print(f"  📝   DETAILS:  {error_msg}")

        print("━" * 60 + "\n")

In [31]:
# Define our user request for the "happy path" scenario
user_request = """
Please process the loan application for Borrower-789.
The loan amount is $250,000.
The application includes the following documents: ['doc_id_123', 'doc_income_456'].
"""

# Call the agent
call_agent(user_request)


>>>> USER REQUEST: Please process the loan application for Borrower-789.
The loan amount is $250,000.
The application includes the following documents: ['doc_id_123', 'doc_income_456'].

  >> [Clock 21:45:52] Initiating request...
--- Agent Activity Log (Filtered) ---
--- Tool Called: validate_document() ---

🧠 THOUGHT:
**Okay, let's get this done.**

Alright, I've got "Borrower-789" with a $250,000 loan application in front of me. Looks like they've submitted documents 'doc_id_123' and 'doc_income_456'. Time to kickstart the loan origination pipeline. 

First things first, I need to make sure these documents are actually all there and that they are what they should be. That's the validation step. Gotta get that foundation solid before we move forward.

Then, I'll need to pull their credit score. That's a crucial piece of the puzzle. With that in hand, I can move on to the risk assessment. The loan amount combined with the credit score will help me figure out the level of risk involve

In [32]:
# Define our user request for the "not so happy path" scenario
user_request = """
Please process the loan application for Borrower-400.
The loan amount is $350,000.
The application includes the following documents: ['doc_id_123', 'doc_income_456'].
"""

# Call the agent
call_agent(user_request)


>>>> USER REQUEST: Please process the loan application for Borrower-400.
The loan amount is $350,000.
The application includes the following documents: ['doc_id_123', 'doc_income_456'].

  >> [Clock 21:46:11] Initiating request...
--- Agent Activity Log (Filtered) ---
--- Tool Called: validate_document() ---

🧠 THOUGHT:
**Loan Application Processing for Borrower-400**

Okay, so I've got this loan application for Borrower-400, a pretty standard request for $350,000.  The system's given me the relevant documentation – 'doc_id_123' and 'doc_income_456'.  Time to get to work.  My immediate thought is to efficiently navigate the loan origination pipeline.  It’s a familiar process: validate the documents, run a credit check, assess the inherent risk, and then make sure everything's compliant.

First step: Let’s call the `validate_document` function with those document IDs.  This will kickstart the critical validation phase and make sure we have everything we need before moving any further. 